# The Full Multi Model exercise

In [ ]:
## Import libraries and functions
##Import libraries
import os
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from scipy.integrate import odeint
import warnings
warnings.filterwarnings('ignore')
##Import functions
from objectGenerationRiver_func import*
from GlobalConstants import * 
from readImputParam import readProcessparam, microplasticData,readCompartmentData
from dilutionVol_calculator_func import*
import RC_GeneratorRiver
from RC_estimation_function import*
from reshape_RC_df_fun2 import*
from fillRCmatrixInteractionsTransport_func import*
from fillRCinteractionMatrices_func import*
from fillInteractions_df_fun import*
from celluloid import Camera
from cycler import cycler
%matplotlib inline
# Load the widgets
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
#Import process parameters
process_df= readProcessparam ("process_paramRiver.txt")
#Import MP parameters (radius, volume, etc) Same way
MP_prop = microplasticData("microplasticsSizeClass.txt")
#Import compartment info
compartments_prop = readCompartmentData("compartmentsGenericRiverSec_prop.txt")
#Add river section depth field
RSdepth = []
for row in range(len(compartments_prop)):
        RSdepth.append(round(sum(compartments_prop.depth_m[0:4]),2))
compartments_prop["depthRS_m"]=RSdepth

In [ ]:
## Model set up
#RIVER COMPARTMENTS
compartments = ["Surface Water", "Flowing Water", "Stagnant Water", "Sediment"]
riverComp = ["1", "2", "3", "4"]

#MICROPLASTICS FORMS 
MPforms = ["A", "B", "C", "D"]
MPformslabels = ["Free", "Heteroaggregated", "Biofiolm-covered", "Biofilm-heteroaggregated"]
#SIZE BINS
sizeBin =["a", "b", "c", "d", "e"]
sizeBinLabel = ["0.1um", "1um","10um", "100um", "1000um"]# Detection limit for MPs via Fourier Transform Infrared Spectroscopy is 20um
#MPS RIVER PROCESSES (FATE AND TRANSPORT) LIST
processList = ["degradation", "fragmentation", "heteroagg", "breakup", "settling","rising", "advection", "mixing", "biofilm", "resusp", "burial","sedTransport", "defouling"]
processLabels = ["Degradation", "Fragmentation", "Heteroaggr", "Heterggr. Breakup", "Settling","Rising", "Advection", "Mixing", "Biofouling", "Resusp.", "Burial","Sed. Transport", "Defouling"]
#RIVER SECTIONS
numberRS=len (compartments_prop)/len(riverComp)
listRS = [*range(0,int(numberRS),1)]
riverSect = [str(item) for item in listRS]
riverLengths = [str(it) for it in compartments_prop["length_m"]]
riverSectLength= riverLengths[0::4]
RS_cumLength_m =[]
for d in range(len(riverSectLength)):
    if d==0:
        RS_cumLength_m.append(float(riverSectLength[d]))
    else:
        RS_cumLength_m.append(float(riverSectLength[d])+float(RS_cumLength_m[d-1]))

In [ ]:
Comp_dic=dict({'Surface Water': "1", 'Flowing Water':"2",'Stagnant Water':"3", "Sediment":"4"})
MPform_dic=dict({'Free': "A", 'Heteroaggregated':"B",'Biofiolm-covered':"C", "Biofilm-heteroaggregated":"C"})
SizeBin_dic=dict({'0.1um': "a", '1um':"b",'10um':"c", "100um":"d","1000um":"e"})
density_dic=dict({"PE": 980, "PA":999, "PVC": 1580})
River_section_dic=dict(zip(listRS, RS_cumLength_m))

In [ ]:
t0 = 0 
daysSimulation = 365
tmax = 24*60*daysSimulation*60 
sec_day = 24*60*60
stepSize= 60*60*24 #time step of 1day
timesteps = int(sec_day*daysSimulation/stepSize) 
t_span = np.linspace(0, tmax, int(timesteps)+1, dtype=int)
from datetime import datetime, timedelta
date_time_str = '2020-01-01 00:00'
DayStart = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
LastDay = DayStart + timedelta(minutes=tmax)
date = DayStart
daterun = date.today()
daterun_label = daterun.strftime("%Y_%m_%d")

In [ ]:
#Generate COMBINATIONS
combinations = list(itertools.product(riverSect,riverComp,MPforms,sizeBin))
#Generate raw list of combinations and lists of concentrations (C) and inflows (I)
CombList = []
Ilist = []
Clist =[]
def convertTuple(tup): 
    str =  ''.join(tup) 
    return str
for e in combinations:
    Clist.append("C_" + convertTuple(e))
    Ilist.append("I_" + convertTuple(e))
    CombList.append(convertTuple(e))

## Define model scenario

In [ ]:
## Define run parameters
SOLVER = "Dynamic" 
mode = "Standard" 
mode2 = "Timelimit" 
record = "True"

In [ ]:
#MP properties selection
#Create drop down menue
style = {'description_width': 'initial'}
MP_type= widgets.Dropdown(
    value='PE',
    placeholder='Select shape',
    options=['PE', 'PA', "PVC"],
    description='Composition:',
    disabled=False, style= style)
#Create text box for float numbers
MP_density= widgets.FloatText(
    value= 910,
    step=1,
    description='Density (kg/m3):',
    disabled=False,style= style)
#Create drop down menue
MP_shape= widgets.Dropdown(
    value='Fragment',
    placeholder='Select shape',
    options=['Fragment', 'Fiber'],
    description='Shape:',
    disabled=False,style= style)

In [ ]:
#Selection input scenario
RC_select= widgets.Dropdown(
    value='Surface Water',
    placeholder='Select compartment',
    options=["Surface Water", "Flowing Water", "Stagnant Water", "Sediment"],
    description='River compartment:',
    disabled=False,style= style)

MP_form= widgets.Dropdown(
    value="Free",
    placeholder='Select shape',
    options=["Free", "Heteroaggregated", "Biofiolm-covered", "Biofilm-heteroaggregated"],
    description='Aggregation state:',
    disabled=False,style= style)

MP_sizeBin=widgets.Dropdown(
    value="1000um",
    placeholder='Select size bin',
    options=["0.1um", "1um","10um", "100um", "1000um"],
    description='Size bin:',
    disabled=False,style= style)

RS_selct = widgets.IntSlider(
    value=0,
    min=0,
    max=19,
    step=1,
    description='River section:',
    disabled=False,style= style)

inputFlow= widgets.FloatText(
    value= 100,
    step=1,
    description='Input flow (No/min):',
    disabled=False,style= style)

inputpulse= widgets.FloatText(
    value= 0,
    step=1,
    description='Input pulse (No/min):',
    disabled=False,style= style)

In [ ]:
MP_properties=widgets.VBox([MP_type,],layout={'width': 'max-content'})
MP_Emissions=widgets.VBox([RC_select,MP_form,MP_sizeBin,RS_selct,inputFlow,inputpulse],layout={'width': 'max-content'})

In [ ]:
#Create a container where to locate widgets.  The widgets that are part of a container widget are called children
#I will use a Tabs container to organise the inputs of the model

children = [MP_properties, MP_Emissions]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'MP properties')
tab.set_title(1, 'MP emissions')

tab

In [ ]:
def imput_transf(RC_select,RS_selct,MP_form,MP_sizeBin,Comp_dic):
    return str(RS_selct.value)+Comp_dic[RC_select.value]+MPform_dic[MP_form.value]+SizeBin_dic[MP_sizeBin.value]


In [ ]:
button_input = widgets.Button(description='Load Scenario')
out_input = widgets.Output(layout={'border': '1px solid black'})
def on_buttonInput_clicked(_):
    out_input.clear_output()
    with out_input:
        composition = MP_type.value
        imputMP=imput_transf(RC_select,RS_selct,MP_form,MP_sizeBin,Comp_dic)
        imputFlow=inputFlow.value
        imputPulse=inputpulse.value
        on_buttonInput_clicked.data=[composition,imputMP,imputFlow,imputPulse]
        dash = '-' * 26
        print('{:<20}'.format("Input parameters"))
        print(dash)
        print('{:<20}'.format("MP composition: " +composition))
        print('{:<20}'.format("MP density: " +str(density_dic[composition])+ " (kg/m3)"))
        print('{:<20}'.format("input code: " +imputMP))
        print('{:<20}'.format("input flow: " +str(imputFlow)+" No/min"))
        print('{:<20}'.format("input pulse: " +str(imputPulse)+" particles"))
        print(dash)
        print('{:<20}'.format("Selected scenario:"))
        print("Emissions of "+ str(inputFlow.value) +" particles per minute of " + MP_type.value + " MPs in ")
        print(MP_form.value + " form of " +MP_sizeBin.value+ " in size, into the "+RC_select.value)
        print("of the riversection "+ str(RS_selct.value))
# linking button and function together using a button's method
button_input.on_click(on_buttonInput_clicked)
# displaying button and its output together
widgets.VBox([button_input,out_input])        

In [ ]:
#Model funcion
def dNdt_2(N,t,k,I):  
    dNdt=np.dot(N,k)+I
    return np.squeeze(dNdt)

In [ ]:
#### Function to extract concentration values by size fraction
def extract_SizeBins (t, comp, MPform,ConcPlot):
    Aa=[]
    Ab=[]
    Ac=[]
    Ad=[]
    Ae=[]
    for i in range(len(listRS)):
        Aa.append(ConcPlot.values[t, Clist.index("C_"+str(listRS[i])+comp+MPform+"a")])
        Ab.append(ConcPlot.values[t, Clist.index("C_"+str(listRS[i])+comp+MPform+"b")])
        Ac.append(ConcPlot.values[t, Clist.index("C_"+str(listRS[i])+comp+MPform+"c")])
        Ad.append(ConcPlot.values[t, Clist.index("C_"+str(listRS[i])+comp+MPform+"d")]) 
        Ae.append(ConcPlot.values[t, Clist.index("C_"+str(listRS[i])+comp+MPform+"e")]) 
    return [Aa, Ab, Ac, Ad, Ae]

In [ ]:
#### Function to extract lists from a list by criteria
def listofindex(criteria,Clist):                                                                                                             
    lista= [[] for x in range(len(criteria))]
    for i in range(len(lista)):
        lista[i] = [n for n in Clist if criteria[i] in n[-3:]]
    return lista

In [ ]:
list_of_indexesMpType=listofindex(MPforms,Clist)
list_of_indexesCompartments=listofindex(riverComp,Clist)
list_ofindexesSizeBins=listofindex(sizeBin,Clist)

In [ ]:
#### Define time resolution for extracting results (time_extract)
numTstep_hour=(60*60/stepSize)
Time_months=t_span[::(int(numTstep_hour*24*30))]
Time_days=t_span[::(int(numTstep_hour*24))]
Time_halfMonth=t_span[::(int(numTstep_hour*24*15))]
Time_5days=t_span[::(int(numTstep_hour*24*5))]#5 days

In [ ]:
time_extract=Time_days

In [ ]:
def plot_all_tfinal(composition,imputMP,imputFlow,imputPulse,ConcPlot,ConcPlot_units):
    import matplotlib.ticker as ticker
    from matplotlib import ticker
    ##Select style
    palette = plt.get_cmap('Set2')
    plt.style.use('seaborn-white')
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='ivory', alpha=0.5)
    props2 = dict(boxstyle='round', facecolor='white', alpha=0.5)
    x =[d/1000 for d in RS_cumLength_m]
    compartmentsLabel=["Surface\n Water", "Flowing\n Water", "Stagnant\n Water", "Sediment"]
    t=len(time_extract)-1
    fig2, axs = plt.subplots(len(compartments),len(MPforms), figsize=(15, 10),sharex='col', sharey= "row", squeeze="True")

    labels = ['0.1 um', '1 um', '10 um', '100 um', '1000 um']
    if imputFlow == 0:
        fig2.suptitle( composition +" plastic particles after "+str(int(time_extract[t]/60/60/24)) + " days (pulse= "+ str(imputPulse)+" particles of 1mm)" , fontsize=18,  y=0.95)

    else:
        fig2.suptitle( composition +" plastic particles after "+str(int(time_extract[t]/60/60/24)) + " days" , fontsize=18,  y=0.95)

    for j in range(len(compartments)):
        if j == 3:
            for k in range(len(MPforms)):
                #Plot
                y = extract_SizeBins (t, riverComp[j], MPforms[k],ConcPlot)
                axs[j,k].plot(x, [e * 10**6/1.3 for e in y[0]], linewidth=2.5,color=palette(0), label='0.1 um')
                axs[j,k].plot(x, [e * 10**6/1.3 for e in y[1]], linewidth=2.5,color=palette(1), label='1 um')
                axs[j,k].plot(x, [e * 10**6/1.3 for e in y[2]], linewidth=2.5,color=palette(2), label='10 um')
                axs[j,k].plot(x, [e * 10**6/1.3 for e in y[3]], linewidth=2.5,color=palette(3), label='100 um')
                axs[j,k].plot(x, [e * 10**6/1.3 for e in y[4]], linewidth=2.5,color=palette(4), label='1000 um')

                if k==3:
                    axs[j,k].text(1.2, 0.5, compartmentsLabel[j] ,fontsize=15,rotation=0, va='center',ha='center', transform=axs[j,k].transAxes)

                axs[j,k].set_yscale('log')
                axs[j,k].set_ylim(10**-9,1000000)
                if k == 0:
                    axs[j,k].set_ylabel("Conc (mg/g)", fontsize=15)
                axs[j,k].yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=4))
                axs[j,k].set_xlim(x[0],x[-1])
                axs[j,k].tick_params(axis='x', labelsize=12, direction='inout', length=6, width=1, grid_alpha=0.5)
                axs[j,k].tick_params(axis='y',labelsize=10, direction='inout', length=6, width=1, grid_alpha=0.5) 
                formatter = ticker.ScalarFormatter(useMathText=True)
                formatter.set_scientific(True) 
                formatter.set_powerlimits((-1,1)) 
        else:
            for k in range(len(MPforms)):
                #Plot
                y = extract_SizeBins (t, riverComp[j], MPforms[k],ConcPlot)
                axs[j,k].plot(x, y[0], linewidth=2.5,color=palette(0), label='0.1 um')
                axs[j,k].plot(x, y[1], linewidth=2.5,color=palette(1), label='1 um')
                axs[j,k].plot(x, y[2], linewidth=2.5,color=palette(2), label='10 um')
                axs[j,k].plot(x, y[3], linewidth=2.5,color=palette(3), label='100 um')
                axs[j,k].plot(x, y[4], linewidth=2.5,color=palette(4), label='1000 um')
                if j== 0:
                    axs[j,k].text(0.5,1.1, MPformslabels[k] ,fontsize=15, transform= axs[j,k].transAxes, ha='center')        
                if k==3:
                    axs[j,k].text(1.2, 0.5, compartmentsLabel[j] ,fontsize=15,rotation=0, va='center',ha='center', transform=axs[j,k].transAxes)
                if k == 0:
                    axs[j,k].set_ylabel("Conc "+ ConcPlot_units[0], fontsize=15)
                axs[j,k].set_yscale('log')

                if j==0:
                    axs[j,k].set_ylim(10**-9,10**1)
                    axs[j,k].yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=4))
                elif j==1:
                    axs[j,k].set_ylim(10**-9,10**1)
                    axs[j,k].yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=4))
                elif j== 2:
                    axs[j,k].set_ylim(10**-9,10**1)
                    axs[j,k].yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=4))
                axs[j,k].set_xlim(x[0],x[-1])

                axs[j,k].tick_params(axis='x', labelsize=10, direction='inout', length=6, width=1, grid_alpha=0.5)
                axs[j,k].tick_params(axis='y',labelsize=10, direction='inout', length=6, width=1, grid_alpha=0.5)
                from matplotlib import ticker
                formatter = ticker.ScalarFormatter(useMathText=True)
                formatter.set_scientific(True) 
                formatter.set_powerlimits((-1,1)) 
                axs[j,k].minorticks_on()


    # Axis titles
    #plt.text(0.02, 0.5, "Concentration of particles (Num/$m^3$)", fontsize=15, transform=plt.gcf().transFigure, rotation='vertical',ha='center', va='center')
    plt.text(0.5, 0.08, "Distance (km)", fontsize=15, transform=plt.gcf().transFigure, ha='center', va='center')
    #plt.legend(labels,bbox_to_anchor=(0.5, -0.18), loc='center',ncol=5, fontsize=15 )
    plt.subplots_adjust(wspace=0.02,hspace=0.1)
    handles, labels = axs[j,k].get_legend_handles_labels()
    fig2.legend(handles, labels, bbox_to_anchor=(0.5, 0.04), loc='center',ncol=5, fontsize=15)
    if imputPulse != 0:
        fig2_label= "ConcvsDist_Multiplot_"+ composition +"_"+ConcPlot_units[1]+"_Pulse.png"
    else:
        fig2_label= "ConcvsDist_Multiplot_"+ composition +"_"+ConcPlot_units[1]+"_ConstantInflow.png"
    return fig2

## Run Model

In [ ]:
button = widgets.Button(description='Run Model')
out = widgets.Output(layout={'border': '1px solid black'})
def on_button_clicked(_):
    out.clear_output()
    with out:
        composition=on_buttonInput_clicked.data[0]
        imputMP=on_buttonInput_clicked.data[1]
        imputFlow=on_buttonInput_clicked.data[2]
        imputPulse=on_buttonInput_clicked.data[3]
        # what happens when we press the button
        RC_df=RC_estimation_function(processList,CombList,Clist,MP_prop,compartments_prop,process_df,numberRS, composition,mode2, mode, date,riverComp,MPforms,sizeBin)
        interactions_df= fillInteractions_fun (RC_df, Clist,compartments_prop)
          #Initial number of particles in the system 
        PartNum_t0 = pd.DataFrame(index=Clist, columns=['number of particles'])
        for p in range(len(PartNum_t0)):
            PartNum_t0.iloc[p][0]= 0
            PartNum_t0.loc["C_"+imputMP]=imputPulse

          #Inflow of particles as particles per second 
        Ilist = []
        for C in Clist:
            Ilist.append("I"+ C[1:])
        inflow_vector = pd.DataFrame(index=Ilist, columns=["number of particles"])
        inflow_vector.loc[:,:] = 0
        inflow_vector.loc["I_"+imputMP] = imputFlow/60 #transformed to particles per sec
          # intitial condition
        N0 = PartNum_t0['number of particles'].to_numpy(dtype="float")
        I= inflow_vector['number of particles'].to_numpy(dtype="float")
          # time points
        time = np.linspace(0, tmax, int(timesteps)+1, dtype=int)##in seconds

        #Solve ODEs
        if SOLVER == 'Dynamic':
            k=interactions_df.to_numpy()
            Nfinal=odeint(dNdt_2, N0, time, args =(k,I), col_deriv=True)
            NFinal_num = pd.DataFrame(data = Nfinal, index=t_span , columns= Clist)  

        elif SOLVER == "SteadyState":
            print("Steady State not yet implemented")

          #Vector of volumes corresponding to the compartments of the river
        dilution_vol_m3= volumesVector(Clist,compartments_prop)

        ConcFinal_num_m3= pd.DataFrame(data = 0, index=t_span , columns= Clist) 
        for ind in range(len(NFinal_num)):
            ConcFinal_num_m3.iloc[ind]=NFinal_num.iloc[ind]/dilution_vol_m3

        #Substitute values smaller than 10-5 to 0
        ConcFinal_num_m3 = ConcFinal_num_m3.apply(lambda x: [y if y >= 1e-15 else 0 for y in x])
        volume= RC_df.loc["volume_m3"].to_numpy()
        density= RC_df.loc["density_kg_m3"].to_numpy()
        ConcFinal_mg_m3=ConcFinal_num_m3*volume*density*10**6
        
        on_button_clicked.data=[ConcFinal_num_m3,ConcFinal_mg_m3]
        print("Model Run sucessfully for "+composition + " MPs. Input scenario: "+imputMP+ " Input flow (No/min) = "+ str(imputFlow)+ "; Input pulse (No) = "+str(imputPulse))
        #display(ConcFinal_num_m3)
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])        

## Display output (choose results units)

In [ ]:
def plot_multiplot(ConcPlot):
    composition=on_buttonInput_clicked.data[0]
    imputMP=on_buttonInput_clicked.data[1]
    imputFlow=on_buttonInput_clicked.data[2]
    imputPulse=on_buttonInput_clicked.data[3]
    #Select Concentration Units: number of particles or mass
    if ConcPlot== "mg/m3":
        ConcPlot = on_button_clicked.data[1] 
        ConcPlot_units= ["(mg/$m^3$)","mg_m3"]
    elif ConcPlot== "Num/m3":
        ConcPlot = on_button_clicked.data[0]
        ConcPlot_units= ["(No/$m^3$)","Num_m3"]
    else:
        print ("Choose correct concentration dataframe")
    fig2= plot_all_tfinal(composition,imputMP,imputFlow,imputPulse,ConcPlot,ConcPlot_units)
    
    return ConcPlot
ConcPlot=widgets.Dropdown(value="mg/m3",
    placeholder='Select concentration units',
    options=["Num/m3", "mg/m3"],
    description='Concentration Units:',
    disabled=False,style= style)
#box = widgets.VBox( ConcPlot )
outPlot = widgets.interactive_output(plot_multiplot, {'ConcPlot':ConcPlot} )
display(ConcPlot, outPlot)
#widgets.VBox([button,out])   